In [ ]:
import pandas as pd
import os
import numpy as np

def merge_gene_drug_files(file1_path, file2_path, output_filename="old.csv", column_order=None):
    """
    Объединяет два файла данных о генах и препаратах с учетом всех требований:
    - Объединение по Gene, UniProt ID и Drug CHEMBL ID
    - Корректная обработка Protein name
    - Правильное заполнение Drug CHEMBL ID
    - Маркировка источника данных
    - Возможность настроить порядок столбцов с помощью column_order
    """
    try:
        # 1. Загрузка данных
        df_chembl = pd.read_csv(file1_path, sep=None, engine='python', encoding='utf-8')
        df_ttd = pd.read_csv(file2_path, sep=None, engine='python', encoding='utf-8')
        
        # 2. Стандартизация названий столбцов
        df_chembl.columns = df_chembl.columns.str.strip()
        df_ttd.columns = df_ttd.columns.str.strip()

        # 3. Подготовка Drug CHEMBL ID
        def get_drug_chembl_id(df):
            for col in ['Drug ChEMBL ID', 'Drug_ID_ChEMBL', 'Drug_CHEMBL_ID', 'Drug CHEMBL ID']:
                if col in df.columns:
                    return df[col].astype(str).replace('nan', 'N/A')
            return pd.Series(['N/A'] * len(df))

        df_chembl['Drug_CHEMBL_ID'] = get_drug_chembl_id(df_chembl)
        df_ttd['Drug_CHEMBL_ID'] = get_drug_chembl_id(df_ttd)

        # 4. Подготовка Protein name
        # Переименовываем столбцы для ясности
        if 'Protein name' in df_chembl.columns:
            df_chembl['Protein_name_chembl'] = df_chembl['Protein name']
        if 'Protein name' in df_ttd.columns:
            df_ttd['Protein_name_ttd'] = df_ttd['Protein name']

        # 5. Добавление временных меток источников
        df_chembl['_source_chembl'] = True
        df_ttd['_source_ttd'] = True

        # 6. Объединение данных по ключевым полям
        merge_keys = ['Gene', 'UniProt ID', 'Drug_CHEMBL_ID']
        merged = pd.merge(
            df_chembl, 
            df_ttd, 
            on=merge_keys, 
            how='outer', 
            suffixes=('_ChEMBL', '_TTD')
        )

        # 7. Обработка Protein name
        merged['Protein name'] = merged['Protein_name_chembl'].combine_first(merged['Protein_name_ttd'])
        
        # 8. Обработка Drug CHEMBL ID
        # Уже правильно объединено по merge_keys

        # 9. Определение источника данных
        conditions = [
            (merged['_source_chembl'].notna() & merged['_source_ttd'].notna()),  # Обе базы
            (merged['_source_chembl'].notna()),  # Только ChEMBL
            (merged['_source_ttd'].notna())      # Только TTD
        ]
        choices = ['ChEMBL & TTD', 'ChEMBL', 'TTD']
        merged['Database'] = np.select(conditions, choices, default='Unknown')

        # 10. Удаление временных и лишних столбцов
        cols_to_drop = ['_source_chembl', '_source_ttd', 'Protein_name_chembl', 'Protein_name_ttd', 
                        'Drug TTD ID', 'Protein TTD ID', 'Protein ChEMBL ID', 'Protein name_ChEMBL', 'Protein name_TTD']
        
        merged.drop(cols_to_drop, axis=1, inplace=True, errors='ignore')

        # 11. Удаление лишнего столбца Drug CHEMBL ID (если появился лишний)
        merged = merged.loc[:, ~merged.columns.str.contains('Drug CHEMBL ID', case=False)].copy()

        # 12. Переименование столбцов
        merged.rename(columns={
            'Drug_CHEMBL_ID': 'Drug ChEMBL ID',
            'Drug name_ChEMBL': 'Drug name ChEMBL',
            'Drug name_TTD': 'Drug name TTD'
        }, inplace=True)

        # 13. Если передан список column_order, применяем его
        if column_order:
            # Проверим, что все столбцы из column_order присутствуют в данных
            if all(col in merged.columns for col in column_order):
                merged = merged[column_order]
            else:
                print("❌ Некоторые столбцы из column_order не найдены в данных, используем стандартный порядок")

        # 14. Сортировка по значению в столбце 'Drug Mechanism ChEMBL'
        # Для начала создаем порядок сортировки для значений "Да", "Нет" и других
        def sort_drug_mechanism(row):
            if row == "Да":
                return 0
            elif row == "Нет":
                return 1
            else:
                return 2

        # Применяем сортировку по "Drug Mechanism ChEMBL"
        merged['Drug Mechanism ChEMBL_sort'] = merged['Drug Mechanism ChEMBL'].apply(sort_drug_mechanism)
        merged = merged.sort_values(by=['Gene', 'Drug Mechanism ChEMBL_sort']).drop(columns=['Drug Mechanism ChEMBL_sort'])

        # 15. Сохранение результата
        output_path = os.path.join(os.getcwd(), output_filename)
        merged.to_csv(output_path, sep=',', index=False, encoding='utf-8')
        
        # 16. Валидация результата
        print(f"✅ Файл успешно сохранен: {output_path}")
        print(f"Всего строк: {len(merged)}")
        print("Распределение по источникам:")
        print(merged['Database'].value_counts())
        
        return merged

    except Exception as e:
        print(f"❌ Ошибка: {str(e)}")
        if 'df_chembl' in locals():
            print("\nСтолбцы в файле ChEMBL:", df_chembl.columns.tolist())
        if 'df_ttd' in locals():
            print("Столбцы в файле TTD:", df_ttd.columns.tolist())
        return None

# Пример использования
if __name__ == '__main__':
    # Укажите пути к вашим файлам
    chembl_file = r"../chembl/extracted_chembl_gene_drug_data.csv"
    ttd_file = r"../ttd/extracted_ttd_gene_drug_data.csv"
    
    # Определите порядок столбцов, если нужно изменить
    custom_column_order = [
        'Gene', 'UniProt ID', 'Protein name',
        'Drug ChEMBL ID', 'Drug name ChEMBL', 'Drug name TTD',
        'Drug Mechanism ChEMBL', 'pChEMBL value',
        'Activity type ChEMBL', 'Activity value ChEMBL',
        'Activity type TTD', 'Activity value TTD',
        'Max phase ChEMBL', 'Max phase TTD',
        'Action type ChEMBL', 'Mechanism of Action ChEMBL', 
        'Action type TTD', 'Database'
    ]
    
    result = merge_gene_drug_files(chembl_file, ttd_file, column_order=custom_column_order)
    
    if result is not None:
        print("\nПервые 5 строк результата:")
        print(result.head())


✅ Файл успешно сохранен: c:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\Новые\Объединение\old.csv
Всего строк: 981
Распределение по источникам:
Database
ChEMBL          905
TTD              68
ChEMBL & TTD      8
Name: count, dtype: int64

Первые 5 строк результата:
   Gene UniProt ID                  Protein name Drug ChEMBL ID  \
2   ALK     Q9UM73  ALK tyrosine kinase receptor  CHEMBL1983268   
13  ALK     Q9UM73  ALK tyrosine kinase receptor  CHEMBL2403108   
35  ALK     Q9UM73  ALK tyrosine kinase receptor  CHEMBL3286830   
40  ALK     Q9UM73  ALK tyrosine kinase receptor  CHEMBL3545311   
41  ALK     Q9UM73  ALK tyrosine kinase receptor  CHEMBL3545360   

   Drug name ChEMBL Drug name TTD Drug Mechanism ChEMBL  pChEMBL value  \
2       ENTRECTINIB   Entrectinib                    Да           9.00   
13        CERITINIB     Ceritinib                    Да           7.39   
35       LORLATINIB           NaN                    Да           9.15   
40       BRIGATINIB           Na

In [1]:
import pandas as pd
import os
import numpy as np
from chembl_webresource_client.new_client import new_client
from typing import List, Dict, Set, Optional, Tuple
import re

# Функции для нормализации и сопоставления названий препаратов
def normalize(text: str) -> str:
    """Радикальная нормализация текста"""
    return re.sub(r"[-–—'\",.(){}\[\]:;!?\\/+\s]", "", text.upper())

def basic_clean(text: str) -> str:
    """Мягкая нормализация"""
    return re.sub(r"[-–—'\",.(){}\[\]:;!?\\/]", "", text.upper())

def get_chembl_family_mapping(chembl_ids: List[str]) -> Dict[str, str]:
    """Создаем mapping: любой ChEMBL ID -> ближайший родительский ID из нашего списка"""
    molecule_form = new_client.molecule_form
    mapping = {}
    
    for chembl_id in chembl_ids:
        mapping[chembl_id] = chembl_id  # Сам себе родитель
        
        try:
            # Получаем всех членов семейства
            family_members = list(molecule_form.filter(parent_chembl_id=chembl_id).only('molecule_chembl_id'))
            for member in family_members:
                if 'molecule_chembl_id' in member:
                    mapping[member['molecule_chembl_id']] = chembl_id
        except Exception as e:
            print(f"Ошибка при получении семейства для {chembl_id}: {e}")
    
    return mapping

def find_drug_matches(ttd_drug_names: List[str], chembl_ids: List[str]) -> Dict[str, str]:
    """Находит соответствия между названиями препаратов из TTD и ChEMBL ID"""
    molecule = new_client.molecule
    matches = {}
    family_mapping = get_chembl_family_mapping(chembl_ids)
    all_related_ids = set(family_mapping.keys())
    
    for drug in ttd_drug_names:
        if pd.isna(drug) or drug.strip() == "":
            continue
            
        found = False
        # 1️⃣ Первая стадия: точный поиск по синонимам
        for chembl_id in all_related_ids:
            try:
                mol = molecule.get(chembl_id)
                synonyms = mol.get("molecule_synonyms", [])
                for syn in synonyms:
                    if isinstance(syn, dict) and drug.lower() == syn.get("molecule_synonym", "").lower():
                        matches[drug] = family_mapping[chembl_id]
                        found = True
                        break
                if found:
                    break
            except Exception:
                continue
        
        if found:
            continue
            
        # 2️⃣ Вторая стадия: поиск в верхнем регистре
        upper_drug = drug.upper()
        for chembl_id in all_related_ids:
            try:
                mol = molecule.get(chembl_id)
                synonyms = mol.get("molecule_synonyms", [])
                for syn in synonyms:
                    if isinstance(syn, dict) and upper_drug == syn.get("molecule_synonym", "").upper():
                        matches[drug] = family_mapping[chembl_id]
                        found = True
                        break
                if found:
                    break
            except Exception:
                continue
        
        if found:
            continue
            
        # 3️⃣ Третья стадия: мягкая нормализация
        cleaned_drug = basic_clean(drug)
        for chembl_id in all_related_ids:
            try:
                mol = molecule.get(chembl_id)
                synonyms = mol.get("molecule_synonyms", [])
                for syn in synonyms:
                    if isinstance(syn, dict) and cleaned_drug == basic_clean(syn.get("molecule_synonym", "")):
                        matches[drug] = family_mapping[chembl_id]
                        found = True
                        break
                if found:
                    break
            except Exception:
                continue
    
    return matches

def merge_gene_drug_files(file1_path, file2_path, output_filename="chembl_ttd_integration_data.csv", column_order=None):
    """
    Объединяет два файла данных о генах и препаратах с учетом всех требований:
    - Объединение по Gene, UniProt ID и Drug CHEMBL ID
    - Объединение по названиям препаратов с использованием ChEMBL API
    - Корректная обработка Drug name
    - Маркировка источника данных
    """
    try:
        # 1. Загрузка данных
        df_chembl = pd.read_csv(file1_path, sep=None, engine='python', encoding='utf-8')
        df_ttd = pd.read_csv(file2_path, sep=None, engine='python', encoding='utf-8')
        
        # 2. Стандартизация названий столбцов
        df_chembl.columns = df_chembl.columns.str.strip()
        df_ttd.columns = df_ttd.columns.str.strip()

        # 3. Подготовка Drug CHEMBL ID (используем старую логику)
        def get_drug_chembl_id(df):
            for col in ['Drug ChEMBL ID', 'Drug_ID_ChEMBL', 'Drug_CHEMBL_ID', 'Drug CHEMBL ID']:
                if col in df.columns:
                    return df[col].astype(str).replace('nan', 'N/A')
            return pd.Series(['N/A'] * len(df))

        df_chembl['Drug_CHEMBL_ID'] = get_drug_chembl_id(df_chembl)
        df_ttd['Drug_CHEMBL_ID'] = get_drug_chembl_id(df_ttd)

        # 4. Получаем все уникальные ChEMBL ID для поиска соответствий
        all_chembl_ids = list(set(df_chembl['Drug_CHEMBL_ID'].unique().tolist() + 
                                df_ttd['Drug_CHEMBL_ID'].unique().tolist()))
        all_chembl_ids = [x for x in all_chembl_ids if x != 'N/A']

        # 5. Находим соответствия между названиями препаратов из TTD и ChEMBL ID
        ttd_drug_names = df_ttd['Drug name'].dropna().unique().tolist()
        drug_matches = find_drug_matches(ttd_drug_names, all_chembl_ids)

        # 6. Создаем временные столбцы для объединения
        df_ttd['Drug_CHEMBL_ID_from_name'] = df_ttd['Drug name'].map(drug_matches)
        
        # 7. Добавляем метки источников
        df_chembl['_source_chembl'] = True
        df_ttd['_source_ttd'] = True

        # 8. Объединение данных по ключевым полям (ChEMBL ID)
        merge_keys = ['Gene', 'UniProt ID', 'Drug_CHEMBL_ID']
        merged = pd.merge(
            df_chembl, 
            df_ttd, 
            on=merge_keys, 
            how='outer', 
            suffixes=('_ChEMBL', '_TTD'),
            indicator='Database'
        )

        # 9. Обновляем столбец Database для понятного отображения источников
        merged['Database'] = merged['Database'].map({
            'left_only': 'ChEMBL',
            'right_only': 'TTD',
            'both': 'ChEMBL & TTD'
        })

        # 10. Объединение по названиям препаратов (где ChEMBL ID не совпали)
        for idx, row in df_ttd.iterrows():
            if pd.notna(row['Drug_CHEMBL_ID_from_name']) and row['Drug_CHEMBL_ID_from_name'] != 'N/A':
                # Ищем строки в df_chembl с таким ChEMBL ID
                chembl_rows = df_chembl[df_chembl['Drug_CHEMBL_ID'] == row['Drug_CHEMBL_ID_from_name']]
                
                for _, chembl_row in chembl_rows.iterrows():
                    # Проверяем, что такая строка еще не объединена
                    merge_condition = (
                        (merged['Gene'] == chembl_row['Gene']) & 
                        (merged['UniProt ID'] == chembl_row['UniProt ID']) & 
                        (merged['Drug_CHEMBL_ID'] == row['Drug_CHEMBL_ID_from_name'])
                    )
                    
                    if not merged[merge_condition].empty:
                        continue
                        
                    # Создаем новую объединенную строку
                    new_row = {
                        'Gene': chembl_row['Gene'],
                        'UniProt ID': chembl_row['UniProt ID'],
                        'Drug_CHEMBL_ID': row['Drug_CHEMBL_ID_from_name'],
                        'Database': 'Оба'
                    }
                    
                    # Заполняем остальные поля из ChEMBL
                    for col in df_chembl.columns:
                        if col not in merge_keys and col != 'Drug_CHEMBL_ID' and col != '_source_chembl':
                            new_col = f"{col}_ChEMBL" if col in df_ttd.columns else col
                            new_row[new_col] = chembl_row[col]
                    
                    # Заполняем остальные поля из TTD
                    for col in df_ttd.columns:
                        if col not in merge_keys and col != 'Drug_CHEMBL_ID' and col != '_source_ttd' and col != 'Drug_CHEMBL_ID_from_name':
                            new_col = f"{col}_TTD" if col in df_chembl.columns else col
                            new_row[new_col] = row[col]
                    
                    merged = pd.concat([merged, pd.DataFrame([new_row])], ignore_index=True)

        # 11. Обработка Protein name
        if 'Protein name_ChEMBL' in merged.columns and 'Protein name_TTD' in merged.columns:
            merged['Protein name'] = merged['Protein name_ChEMBL'].combine_first(merged['Protein name_TTD'])
            merged.drop(['Protein name_ChEMBL', 'Protein name_TTD'], axis=1, inplace=True, errors='ignore')
        elif 'Protein name_ChEMBL' in merged.columns:
            merged.rename(columns={'Protein name_ChEMBL': 'Protein name'}, inplace=True)
        elif 'Protein name_TTD' in merged.columns:
            merged.rename(columns={'Protein name_TTD': 'Protein name'}, inplace=True)

        # 12. Обработка Drug name
        if 'Drug name_ChEMBL' in merged.columns and 'Drug name_TTD' in merged.columns:
            merged['Drug name'] = merged['Drug name_ChEMBL'].combine_first(merged['Drug name_TTD'])
            merged.drop(['Drug name_ChEMBL', 'Drug name_TTD'], axis=1, inplace=True, errors='ignore')
        elif 'Drug name_ChEMBL' in merged.columns:
            merged.rename(columns={'Drug name_ChEMBL': 'Drug name'}, inplace=True)
        elif 'Drug name_TTD' in merged.columns:
            merged.rename(columns={'Drug name_TTD': 'Drug name'}, inplace=True)

        # 13. Удаление временных и лишних столбцов
        cols_to_drop = ['_source_chembl', '_source_ttd', 'Drug_CHEMBL_ID_from_name',
                       'Drug TTD ID', 'Protein TTD ID', 'Protein ChEMBL ID']
        
        merged.drop([col for col in cols_to_drop if col in merged.columns], axis=1, inplace=True, errors='ignore')

        # 14. Удаление дубликатов столбцов Drug CHEMBL ID (из старой версии)
        merged = merged.loc[:, ~merged.columns.duplicated()].copy()
        merged = merged.loc[:, ~merged.columns.str.contains('^Drug CHEMBL ID$', case=False, regex=True)].copy()
        
        # 15. Переименовываем Drug_CHEMBL_ID в Drug ChEMBL ID (из старой версии)
        merged.rename(columns={'Drug_CHEMBL_ID': 'Drug ChEMBL ID'}, inplace=True)

        # 16. Если передан список column_order, применяем его
        if column_order:
            # Проверим, что все столбцы из column_order присутствуют в данных
            existing_cols = [col for col in column_order if col in merged.columns]
            other_cols = [col for col in merged.columns if col not in existing_cols]
            merged = merged[existing_cols + other_cols]

        # 17. Сортировка по значению в столбце 'Drug Mechanism ChEMBL'
        if 'Drug Mechanism ChEMBL' in merged.columns:
            def sort_drug_mechanism(row):
                if row == "Да":
                    return 0
                elif row == "Нет":
                    return 1
                else:
                    return 2

            merged['Drug Mechanism ChEMBL_sort'] = merged['Drug Mechanism ChEMBL'].apply(sort_drug_mechanism)
            merged = merged.sort_values(by=['Gene', 'Drug Mechanism ChEMBL_sort'])
            merged.drop('Drug Mechanism ChEMBL_sort', axis=1, inplace=True)

        # 18. Сохранение результата
        output_path = os.path.join(os.getcwd(), output_filename)
        merged.to_csv(output_path, sep=',', index=False, encoding='utf-8')
        
        # 19. Валидация результата
        print(f"✅ Файл успешно сохранен: {output_path}")
        print(f"Всего строк: {len(merged)}")
        print("Распределение по источникам:")
        print(merged['Database'].value_counts())
        
        return merged

    except Exception as e:
        print(f"❌ Ошибка: {str(e)}")
        if 'df_chembl' in locals():
            print("\nСтолбцы в файле ChEMBL:", df_chembl.columns.tolist())
        if 'df_ttd' in locals():
            print("Столбцы в файле TTD:", df_ttd.columns.tolist())
        return None

# Пример использования
if __name__ == '__main__':
    # Укажите пути к вашим файлам
    chembl_file = r"../chembl/extracted_chembl_gene_drug_data.csv"
    ttd_file = r"../ttd/extracted_ttd_gene_drug_data.csv"
    
    # Определите порядок столбцов, если нужно изменить
    custom_column_order = [
        'Gene', 'UniProt ID', 'Protein name',
        'Drug ChEMBL ID', 'Drug name',
        'Drug Mechanism ChEMBL', 'pChEMBL value',
        'Activity type ChEMBL', 'Activity value ChEMBL',
        'Activity type TTD', 'Activity value TTD',
        'Max phase ChEMBL', 'Max phase TTD',
        'Action type ChEMBL', 'Mechanism of Action ChEMBL', 
        'Action type TTD', 'Database'
    ]
    
    result = merge_gene_drug_files(chembl_file, ttd_file, column_order=custom_column_order)
    
    if result is not None:
        print("\nПервые 5 строк результата:")
        print(result.head())

✅ Файл успешно сохранен: c:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\data_integration\chembl_ttd_integration_data.csv
Всего строк: 211
Распределение по источникам:
Database
ChEMBL          183
TTD              23
ChEMBL & TTD      5
Name: count, dtype: int64

Первые 5 строк результата:
   Gene UniProt ID                  Protein name Drug ChEMBL ID    Drug name  \
2   ALK     Q9UM73  ALK tyrosine kinase receptor  CHEMBL1983268  ENTRECTINIB   
13  ALK     Q9UM73  ALK tyrosine kinase receptor  CHEMBL2403108    CERITINIB   
35  ALK     Q9UM73  ALK tyrosine kinase receptor  CHEMBL3286830   LORLATINIB   
40  ALK     Q9UM73  ALK tyrosine kinase receptor  CHEMBL3545311   BRIGATINIB   
41  ALK     Q9UM73  ALK tyrosine kinase receptor  CHEMBL3545360     ASP-3026   

   Drug Mechanism ChEMBL  pChEMBL value Activity type ChEMBL  \
2                     Да           9.00                   Ki   
13                    Да           7.39                 IC50   
35              

In [6]:
import pandas as pd
import os
import numpy as np
from chembl_webresource_client.new_client import new_client
from typing import List, Dict, Set, Optional, Tuple
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import lru_cache
from tqdm import tqdm
import time
import warnings

# Настройки
warnings.filterwarnings('ignore')
API_DELAY = 0.2
MAX_WORKERS = 2
CACHE_SIZE = 1000
BATCH_SIZE = 50

# Функции нормализации
def normalize(text: str) -> str:
    return re.sub(r"[-–—'\",.(){}\[\]:;!?\\/+\s]", "", text.upper())

def basic_clean(text: str) -> str:
    return re.sub(r"[-–—'\",.(){}\[\]:;!?\\/]", "", text.upper())

# Кэшированные запросы
@lru_cache(maxsize=CACHE_SIZE)
def get_molecule_cached(chembl_id: str) -> Optional[dict]:
    time.sleep(API_DELAY)
    try:
        return new_client.molecule.get(chembl_id)
    except Exception as e:
        print(f"Ошибка при получении {chembl_id}: {str(e)}")
        return None

def get_molecules_batch(chembl_ids: List[str]) -> Dict[str, Optional[dict]]:
    results = {}
    for i in range(0, len(chembl_ids), BATCH_SIZE):
        batch = chembl_ids[i:i+BATCH_SIZE]
        try:
            molecules = new_client.molecule.filter(molecule_chembl_id__in=batch).only([
                'molecule_chembl_id', 'molecule_synonyms', 'parent_chembl_id'
            ])
            for mol in molecules:
                results[mol['molecule_chembl_id']] = mol
            time.sleep(API_DELAY * 2)
        except Exception as e:
            print(f"Ошибка при получении пачки: {str(e)}")
            for chembl_id in batch:
                results[chembl_id] = get_molecule_cached(chembl_id)
    return results

def get_chembl_family_mapping_safe(chembl_ids: List[str]) -> Dict[str, str]:
    mapping = {}
    for chembl_id in tqdm(chembl_ids, desc="Building family map"):
        mapping[chembl_id] = chembl_id
        try:
            time.sleep(API_DELAY)
            family_members = list(new_client.molecule_form.filter(parent_chembl_id=chembl_id).only('molecule_chembl_id'))
            for member in family_members:
                if 'molecule_chembl_id' in member:
                    mapping[member['molecule_chembl_id']] = chembl_id
        except Exception as e:
            print(f"Ошибка при получении семейства для {chembl_id}: {str(e)}")
    return mapping

def find_drug_matches_safe(ttd_drug_names: List[str], chembl_ids: List[str]) -> Dict[str, str]:
    if not chembl_ids or not ttd_drug_names:
        return {}
    
    matches = {}
    chunk_size = 100
    
    for i in tqdm(range(0, len(chembl_ids), chunk_size), desc="Processing ChEMBL IDs"):
        chunk_ids = chembl_ids[i:i+chunk_size]
        molecules_data = get_molecules_batch(chunk_ids)
        family_mapping = get_chembl_family_mapping_safe(chunk_ids)
        
        synonyms_index = {}
        for chembl_id, mol_data in molecules_data.items():
            if not mol_data or 'molecule_synonyms' not in mol_data:
                continue
                
            for syn in mol_data['molecule_synonyms']:
                if not isinstance(syn, dict):
                    continue
                    
                syn_text = syn.get("molecule_synonym", "")
                if not syn_text:
                    continue
                    
                variants = {syn_text.lower(), syn_text.upper(), basic_clean(syn_text)}
                
                for variant in variants:
                    if variant not in synonyms_index:
                        synonyms_index[variant] = chembl_id
        
        for drug in ttd_drug_names:
            if pd.isna(drug) or not str(drug).strip() or drug in matches:
                continue
                
            for variant in [str(drug).lower(), str(drug).upper(), basic_clean(str(drug))]:
                if variant in synonyms_index:
                    chembl_id = synonyms_index[variant]
                    matches[drug] = family_mapping.get(chembl_id, chembl_id)
                    break
    
    return matches

def remove_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    """Удаление дубликатов с приоритетом на 'ChEMBL & TTD'"""
    if df.empty:
        return df
    
    # Создаем столбец для сортировки (приоритет: 'ChEMBL & TTD' > 'ChEMBL' > 'TTD')
    df['_sort_priority'] = df['Database'].map({
        'ChEMBL & TTD': 0,
        'ChEMBL': 1,
        'TTD': 2
    })
    
    # Сортируем по приоритету и удаляем дубликаты
    df = df.sort_values(by=['Gene', 'Drug ChEMBL ID', '_sort_priority'])
    df = df.drop_duplicates(subset=['Gene', 'Drug ChEMBL ID'], keep='first')
    
    # Удаляем временный столбец
    df = df.drop(columns=['_sort_priority'])
    
    return df

def safe_merge_data(df_chembl: pd.DataFrame, df_ttd: pd.DataFrame) -> pd.DataFrame:
    try:
        merge_keys = ['Gene', 'UniProt ID', 'Drug_CHEMBL_ID']
        
        df_chembl['_source_chembl'] = True
        df_ttd['_source_ttd'] = True
        
        merged = pd.merge(
            df_chembl, 
            df_ttd, 
            on=merge_keys, 
            how='outer', 
            suffixes=('_ChEMBL', '_TTD'),
            indicator='Database'
        )
        
        merged['Database'] = merged['Database'].map({
            'left_only': 'ChEMBL',
            'right_only': 'TTD',
            'both': 'ChEMBL & TTD'
        })
        
        return merged
        
    except Exception as e:
        print(f"Ошибка при объединении данных: {str(e)}")
        return pd.DataFrame()

def merge_gene_drug_files_safe(file1_path, file2_path, output_filename, column_order=None):
    try:
        # 1. Загрузка данных
        dtype_spec = {
            'Gene': 'string',
            'UniProt ID': 'string',
            'Drug ChEMBL ID': 'string',
            'Drug_ID_ChEMBL': 'string',
            'Drug_CHEMBL_ID': 'string',
            'Drug name': 'string',
            'Protein name': 'string'
        }
        
        df_chembl = pd.read_csv(file1_path, sep=None, engine='python', 
                              encoding='utf-8', dtype=dtype_spec, na_values=['NA', 'N/A', ''])
        df_ttd = pd.read_csv(file2_path, sep=None, engine='python',
                           encoding='utf-8', dtype=dtype_spec, na_values=['NA', 'N/A', ''])

        # 2. Стандартизация столбцов
        df_chembl.columns = df_chembl.columns.str.strip()
        df_ttd.columns = df_ttd.columns.str.strip()

        # 3. Подготовка Drug CHEMBL ID
        def get_drug_chembl_id(df):
            for col in ['Drug ChEMBL ID', 'Drug_ID_ChEMBL', 'Drug_CHEMBL_ID', 'Drug CHEMBL ID']:
                if col in df.columns:
                    return df[col].astype('string').replace(['nan', 'NA', 'N/A'], pd.NA)
            return pd.Series([pd.NA] * len(df), dtype='string')

        df_chembl['Drug_CHEMBL_ID'] = get_drug_chembl_id(df_chembl)
        df_ttd['Drug_CHEMBL_ID'] = get_drug_chembl_id(df_ttd)

        # 4. Получаем уникальные ChEMBL ID
        chembl_ids = pd.concat([
            df_chembl['Drug_CHEMBL_ID'],
            df_ttd['Drug_CHEMBL_ID']
        ]).dropna().unique().tolist()
        
        # 5. Поиск соответствий
        ttd_drug_names = df_ttd['Drug name'].dropna().unique().tolist()
        if chembl_ids and ttd_drug_names:
            print("Начало поиска соответствий препаратов...")
            drug_matches = find_drug_matches_safe(ttd_drug_names, chembl_ids)
            df_ttd['Drug_CHEMBL_ID_from_name'] = df_ttd['Drug name'].map(drug_matches)
        else:
            df_ttd['Drug_CHEMBL_ID_from_name'] = pd.NA
        
        # 6. Основное объединение
        print("Объединение данных...")
        merged = safe_merge_data(df_chembl, df_ttd)
        if merged.empty:
            print("Ошибка: не удалось объединить данные")
            return None

        # 7. Дополнительное объединение по названиям препаратов
        if 'Drug_CHEMBL_ID_from_name' in df_ttd.columns:
            name_matches = df_ttd[df_ttd['Drug_CHEMBL_ID_from_name'].notna()]
            if not name_matches.empty:
                chembl_matches = df_chembl[df_chembl['Drug_CHEMBL_ID'].isin(name_matches['Drug_CHEMBL_ID_from_name'])]
                
                if not chembl_matches.empty:
                    temp_merged = pd.merge(
                        chembl_matches,
                        name_matches,
                        left_on=['Gene', 'UniProt ID', 'Drug_CHEMBL_ID'],
                        right_on=['Gene', 'UniProt ID', 'Drug_CHEMBL_ID_from_name'],
                        how='inner',
                        suffixes=('_ChEMBL', '_TTD')
                    )
                    temp_merged['Database'] = 'ChEMBL & TTD'
                    merged = pd.concat([merged, temp_merged], ignore_index=True)

        # 8. Удаление дубликатов перед объединением столбцов
        print("Удаление дубликатов...")
        merged = remove_duplicates(merged)

        # 9. Объединение столбцов
        def combine_columns(prefix):
            col1, col2 = f'{prefix}_ChEMBL', f'{prefix}_TTD'
            if col1 in merged.columns and col2 in merged.columns:
                merged[prefix] = merged[col1].combine_first(merged[col2])
                merged.drop([col1, col2], axis=1, inplace=True)
            elif col1 in merged.columns:
                merged.rename(columns={col1: prefix}, inplace=True)
            elif col2 in merged.columns:
                merged.rename(columns={col2: prefix}, inplace=True)

        for col in ['Protein name', 'Drug name']:
            combine_columns(col)

        # 10. Удаление временных столбцов
        cols_to_drop = ['_source_chembl', '_source_ttd', 'Drug_CHEMBL_ID_from_name',
                       'Drug TTD ID', 'Protein TTD ID', 'Protein ChEMBL ID']
        merged.drop(columns=[col for col in cols_to_drop if col in merged.columns], inplace=True)

        # 11. Переименование и сортировка
        merged.rename(columns={'Drug_CHEMBL_ID': 'Drug ChEMBL ID'}, inplace=True)

        if column_order:
            existing_cols = [col for col in column_order if col in merged.columns]
            other_cols = [col for col in merged.columns if col not in existing_cols]
            merged = merged[existing_cols + other_cols]

        if 'Drug Mechanism ChEMBL' in merged.columns:
            sort_order = {"Yes": 0, "No": 1, None: 2}
            merged['_sort_temp'] = merged['Drug Mechanism ChEMBL'].map(sort_order)
            merged.sort_values(['Gene', '_sort_temp'], inplace=True)
            merged.drop('_sort_temp', axis=1, inplace=True)

        # 12. Сохранение результата
        os.makedirs(os.path.dirname(output_filename) or '.', exist_ok=True)
        merged.to_csv(output_filename, index=False, encoding='utf-8')
        print(f"Файл успешно сохранен: {output_filename}")
        return merged

    except Exception as e:
        print(f"❌ Критическая ошибка при обработке файлов: {str(e)}")
        return None

def process_files_safely(file_pairs: List[Tuple[str, str, str]], column_order: List[str]):
    for chembl_path, ttd_path, output_filename in file_pairs:
        print(f"\n🔍 Начата обработка файлов:")
        print(f"ChEMBL: {chembl_path}")
        print(f"TTD: {ttd_path}")
        
        result = merge_gene_drug_files_safe(
            chembl_path, 
            ttd_path, 
            output_filename, 
            column_order=column_order
        )
        
        if result is not None:
            print(f"✅ Успешно обработан и сохранен как {output_filename}")
            print(f"Всего строк: {len(result)}")
            print("Распределение по источникам:")
            print(result['Database'].value_counts())
        else:
            print(f"❌ Не удалось обработать файлы: {chembl_path}, {ttd_path}")

def process_all_files():
    chembl_dir = r"C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\chembl\Результаты расчёта"
    ttd_dir = r"C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\ttd\Результаты"
    output_dir = os.path.join(os.getcwd(), "Results")
    
    os.makedirs(output_dir, exist_ok=True)
    
    chembl_files = {f for f in os.listdir(chembl_dir) if f.endswith('.csv')}
    ttd_files = {f for f in os.listdir(ttd_dir) if f.endswith('.csv')}
    common_files = chembl_files & ttd_files
    
    if not common_files:
        print("❌ Нет общих файлов для обработки")
        return
    
    column_order = [
        'Gene', 'UniProt ID', 'Protein name',
        'Drug ChEMBL ID', 'Drug name',
        'Drug Mechanism ChEMBL', 'pChEMBL value',
        'Activity type ChEMBL', 'Activity value ChEMBL',
        'Activity type TTD', 'Activity value TTD',
        'Max phase ChEMBL', 'Max phase TTD',
        'Action type ChEMBL', 'Mechanism of Action ChEMBL', 
        'Action type TTD', 'Database'
    ]
    
    file_pairs = []
    for filename in common_files:
        chembl_path = os.path.join(chembl_dir, filename)
        ttd_path = os.path.join(ttd_dir, filename)
        output_filename = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}.integrated.csv")
        file_pairs.append((chembl_path, ttd_path, output_filename))
    
    process_files_safely(file_pairs, column_order)

if __name__ == '__main__':
    process_all_files()


🔍 Начата обработка файлов:
ChEMBL: C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\chembl\Результаты расчёта\ALL_depscore_-0.15_zscore_-1.csv
TTD: C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\ttd\Результаты\ALL_depscore_-0.15_zscore_-1.csv
Начало поиска соответствий препаратов...


Processing ChEMBL IDs: 100%|██████████| 32/32 [11:29<00:00, 21.56s/it]


Объединение данных...
Удаление дубликатов...
Файл успешно сохранен: c:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\data_integration\Results\ALL_depscore_-0.15_zscore_-1.integrated.csv
✅ Успешно обработан и сохранен как c:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\data_integration\Results\ALL_depscore_-0.15_zscore_-1.integrated.csv
Всего строк: 3094
Распределение по источникам:
Database
ChEMBL          2953
ChEMBL & TTD      87
TTD               54
Name: count, dtype: int64

🔍 Начата обработка файлов:
ChEMBL: C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\chembl\Результаты расчёта\ALL_depscore_-0.15_zscore_-0.5.csv
TTD: C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\ttd\Результаты\ALL_depscore_-0.15_zscore_-0.5.csv
Начало поиска соответствий препаратов...


Processing ChEMBL IDs: 100%|██████████| 96/96 [36:03<00:00, 22.54s/it]


Объединение данных...
Удаление дубликатов...
Файл успешно сохранен: c:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\data_integration\Results\ALL_depscore_-0.15_zscore_-0.5.integrated.csv
✅ Успешно обработан и сохранен как c:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\data_integration\Results\ALL_depscore_-0.15_zscore_-0.5.integrated.csv
Всего строк: 10355
Распределение по источникам:
Database
ChEMBL          9985
ChEMBL & TTD     202
TTD              168
Name: count, dtype: int64

🔍 Начата обработка файлов:
ChEMBL: C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\chembl\Результаты расчёта\AML_depscore_-0.15_zscore_-0.5.csv
TTD: C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\ttd\Результаты\AML_depscore_-0.15_zscore_-0.5.csv
Начало поиска соответствий препаратов...


...

Объединение данных...
Удаление дубликатов...
Файл успешно сохранен: c:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\data_integration\Results\ALL_depscore_-0.2_zscore_-1.integrated.csv
✅ Успешно обработан и сохранен как c:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\data_integration\Results\ALL_depscore_-0.2_zscore_-1.integrated.csv
Всего строк: 2676
Распределение по источникам:
Database
ChEMBL          2544
ChEMBL & TTD      84
TTD               48
Name: count, dtype: int64

🔍 Начата обработка файлов:
ChEMBL: C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\chembl\Результаты расчёта\AML_depscore_-0.2_zscore_-0.5.csv
TTD: C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\ttd\Результаты\AML_depscore_-0.2_zscore_-0.5.csv
Начало поиска соответствий препаратов...


Processing ChEMBL IDs: 100%|██████████| 30/30 [10:54<00:00, 21.82s/it]


Объединение данных...
Удаление дубликатов...
Файл успешно сохранен: c:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\data_integration\Results\AML_depscore_-0.2_zscore_-0.5.integrated.csv
✅ Успешно обработан и сохранен как c:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\data_integration\Results\AML_depscore_-0.2_zscore_-0.5.integrated.csv
Всего строк: 3358
Распределение по источникам:
Database
ChEMBL          3276
ChEMBL & TTD      42
TTD               40
Name: count, dtype: int64

🔍 Начата обработка файлов:
ChEMBL: C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\chembl\Результаты расчёта\AML_depscore_-0.2_zscore_-1.5.csv
TTD: C:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\ttd\Результаты\AML_depscore_-0.2_zscore_-1.5.csv
Начало поиска соответствий препаратов...


Processing ChEMBL IDs: 100%|██████████| 4/4 [01:15<00:00, 18.90s/it]

Объединение данных...
Удаление дубликатов...
Файл успешно сохранен: c:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\data_integration\Results\AML_depscore_-0.2_zscore_-1.5.integrated.csv
✅ Успешно обработан и сохранен как c:\Users\rusla\OneDrive\Рабочий стол\Диплом\Гены\chembl_uniprot_ttd\data_integration\Results\AML_depscore_-0.2_zscore_-1.5.integrated.csv
Всего строк: 344
Распределение по источникам:
Database
ChEMBL          326
ChEMBL & TTD     11
TTD               7
Name: count, dtype: int64
